In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
root_loc = '/Volumes/workspace/default/chase_data'

schema = StructType([
    StructField("Transaction Date", StringType(), False),
    StructField("Post Date",StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Type", StringType(), True),
    StructField("Amount", DoubleType(), True),
    StructField("Memo", StringType(), True)
    ])

In [0]:
df = (
  spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("rescuedDataColumn", "_rescued_data")
    .option("header", "true")
    .schema(schema)
    .load(root_loc)
)
df_clean = df.selectExpr(
    "`Transaction Date` as Transaction_Date",
    "`Post Date` as Post_Date",
    "Description",
    'Category',
    'Type',
    'Amount',
    'Memo',
    '_rescued_data'
)

In [0]:
(
  df_clean.writeStream
    .option("checkpointLocation", "/Volumes/workspace/default/checkpoints/chase_data_checkpoint")
    .trigger(availableNow=True)
    .toTable("workspace.default.chase_raw_data")
)